## 사용자 기반 협업 필터링

특정 사용자 A와 유사한 다른 사용자 B를 찾은 후, B의 관심사를 추천

In [44]:
from numpy import dot
from math import sqrt
from collections import defaultdict

In [45]:
users_interests = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["Hadoop", "Big Data", "HBase", "Java"],
    ["Hadoop", "Big Data", "HBase", "Java", "Postgres", "pandas", "MySQL"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

In [46]:
# user_interests의 항목을 unique한 리스트로 구하기
unique_interests = sorted(list({ interest
                                 for user_interests in users_interests
                                 for interest in user_interests }))

In [47]:
# user_interests의 항목을 unique와 비교하여 벡터로 만들기
def make_user_interest_vector(user_interests):
    """unique_interest[i]가 user_interests에 존재한다면
        i번째 요소가 1이고, 존재하지 않는다면 0인 벡터를 생성"""
    return [1 if interest in user_interests else 0
            for interest in unique_interests]

# 모든 user_interests의 벡터 행렬
user_interest_matrix = list(map(make_user_interest_vector, users_interests))

In [48]:
# 벡터의 코사인 유사도 함수
def cosine_similarity(v, w):
    return dot(v, w) / sqrt(dot(v, v) * dot(w, w))

cosine similarity는 두 벡터 v, w 사이의 각도를 잰다. 완전히 같은 방향이면 1, 완전히 반대 방향이면 -1, v, w 중 하나가 0이면 0을 나타낸다.

In [49]:
# 모든 벡터 2개 조합의 코사인 유사도 구하기
user_similarities = [[cosine_similarity(interest_vector_i, interest_vector_j)
                      for interest_vector_j in user_interest_matrix]
                     for interest_vector_i in user_interest_matrix]

In [50]:
# user_id와 가장 유사한 other_user_id 구하기
def most_similar_users_to(user_id):
    pairs = [(other_user_id, similarity)
             for other_user_id, similarity in
                enumerate(user_similarities[user_id])         # user_id와 다른 모든 유저의 similarity
             if user_id != other_user_id and similarity > 0]  # id가 같지 않고, 유사도가 0이 아닌 것

    return sorted(pairs, key=lambda pair: pair[1], reverse=True)    # similarity가 높은 순서대로 출력

각각의 관심사에 대해 해당 관심사에 관심이 있는 다른 사용자와의 유사도를 모두 더해 새로운 관심사를 추천한다

In [51]:
def user_based_suggestions(user_id, include_current_interests=False):
    
    suggestions = defaultdict(float)
    
    for other_user_id, similarity in most_similar_users_to(user_id):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity    # 다른 유저의 관심사의 similarity를 모두 더한다

    # 정렬된 리스트로 변환
    suggestions = sorted(suggestions.items(), key=lambda pair: pair[1], reverse=True)

    # 이미 관심사로 표시한 것은 제외
    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]

In [52]:
if __name__ == "__main__":
    
    print("User based similarity")
    print("most similar to 0")
    print(most_similar_users_to(0))

    print("Suggestions for 0")
    print(user_based_suggestions(0))
    print()

User based similarity
most similar to 0
[(1, 1.0), (2, 0.7559289460184544), (3, 0.5714285714285714), (12, 0.5669467095138409), (4, 0.3380617018914066), (11, 0.1889822365046136), (16, 0.1690308509457033), (8, 0.1543033499620919)]
Suggestions for 0
[('Postgres', 1.0785211242656814), ('MySQL', 0.7404594223742746), ('pandas', 0.5714285714285714), ('MapReduce', 0.5669467095138409), ('MongoDB', 0.50709255283711), ('NoSQL', 0.3380617018914066), ('neural networks', 0.1889822365046136), ('deep learning', 0.1889822365046136), ('artificial intelligence', 0.1889822365046136), ('databases', 0.1690308509457033), ('Python', 0.1543033499620919), ('R', 0.1543033499620919), ('C++', 0.1543033499620919), ('Haskell', 0.1543033499620919), ('programming languages', 0.1543033499620919)]



차원이 아주 커지면, 대부분의 벡터는 서로 상당히 다른 방향을 가리키게 된다.

즉, 관심사의 수가 아주 많아지면 특정 사용자와 가장 유사한 사용자는 전혀 유사하지 않을 가능성이 있다.